In [ ]:
!pip install transformers sentencepiece tokenizers datasets simplet5 pipeline
!pip install -q "tensorflow-text==2.8.*"

In [ ]:
from transformers import pipeline
import datasets
import tensorflow as tf
import tensorflow_text as text
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
dataset = datasets.load_dataset(
    'GroNLP/ik-nlp-22_slp', 'questions'
)
model_name = "deepset/roberta-base-squad2-distilled"
qa = pipeline(model=model_name, tokenizer=model_name, task="question-answering")

Reusing dataset ik_nlp22_slp (/root/.cache/huggingface/datasets/GroNLP___ik_nlp22_slp/questions/1.0.0/6c89281b2028a8a126102dda2c3fb94b1a5ccea59943d26857ae138c7aa782f8)


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
generated_answers = []
for i,j in zip(dataset['test']['question'],dataset['test']['paragraph']):
  generated_answers.append(qa({'question':i,'context':j})['answer'])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:299: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  for span_id in range(num_spans)


In [ ]:
# Preprocessing 
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
predictions = [normalize_text(s) for s in generated_answers]
predictions = tf.ragged.constant([word_tokenize(s) for s in predictions])

gold = [normalize_text(s) for s in dataset['test']['answer']]
gold = tf.ragged.constant([word_tokenize(s) for s in gold])

In [ ]:
result = text.metrics.rouge_l(predictions, gold)
print('F-Measure: %s' % result.f_measure)
print('P-Measure: %s' % result.p_measure)
print('R-Measure: %s' % result.r_measure)

In [ ]:
result = text.metrics.rouge_l(predictions, gold)
print('Precision: ', float(sum(result.p_measure)/len(result.p_measure)))
print('Recall: ', float(sum(result.r_measure)/len(result.r_measure)))
print('F1: ', float(sum(result.f_measure)/len(result.f_measure)))

Precision:  0.923540472984314
Recall:  0.5673647522926331
F1:  0.6566293239593506
